# Introducción

El siguiente *notebook* contiene el preprocesamiento y procesamiento de la base de datos M7.EV.DS - Dataset.xlsx provista por el equipo docente del diplomado de Data Science de la Universidad de Santiago de Chile,  para la realización del trabajo final del Módulo 7 "Fundamentos de Visualización de datos" del diplomado.

Alumno: Martín Bonnefoy Valdés

# Librerías ocupadas

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans

# Preprocesamiento de datos

In [2]:
path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSDxj5tB3_yGcpvI6d0tEdtAkl0UK11xiw2V6AOvzaf_empxsPPIZzLXz7QnRPk7g/pub?output=xlsx'
# Base de datos es subida y publicada en Google Drive para acceder a ella sin necesitar cargarla manualmente

df = pd.read_excel(path)

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365.0,71053.0,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365.0,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Para revisar la base de datos se busca detectar y eliminar:

- Filas donde hubieran valores duplicados.
- Filas donde hubieran valores anómalos.
- Filas donde hubieran valores perdidos (NaN)

In [5]:
# Se busca identificar las filas donde hubieran valores duplicados

df.duplicated().sum()

5268

In [6]:
# Se eliminan las filas con valores duplicados (5268 filas)

df = df.drop_duplicates()

In [7]:
# Se busca identificar filas donde el valor de UnitPrice fuese menor a 0

df.loc[df['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1.0,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1.0,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [8]:
# Asimismo, se observa una serie de casos donde UnitPrice es igual a 0

df.loc[df['UnitPrice'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414.0,22139.0,NaN,56.0,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545.0,21134.0,NaN,1.0,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546.0,22145.0,NaN,1.0,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547.0,37509.0,NaN,1.0,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549.0,85226A,NaN,1.0,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
536981,581234.0,72817.0,NaN,27.0,2011-12-08 10:33:00,0.0,NaN,United Kingdom
538504,581406.0,46000M,POLYESTER FILLER PAD 45x45cm,240.0,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538505,581406.0,46000S,POLYESTER FILLER PAD 40x40cm,300.0,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538554,581408.0,85175.0,NaN,20.0,2011-12-08 14:06:00,0.0,NaN,United Kingdom


In [9]:
# Inicialmente se considera generar una fucnión que permita reemplazar los valores
# faltantes de UnitPrice a partir de identificarlos en otras columnas

In [10]:
df.loc[df['StockCode'] == 22960]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
21,536368.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 08:34:00,4.25,13047.0,United Kingdom
170,536385.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 09:56:00,4.25,17420.0,United Kingdom
212,536390.0,22960.0,JAM MAKING SET WITH JARS,12.0,2010-12-01 10:19:00,3.75,17511.0,United Kingdom
856,536464.0,22960.0,JAM MAKING SET WITH JARS,1.0,2010-12-01 12:23:00,4.25,17968.0,United Kingdom
866,536466.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 12:23:00,4.25,14849.0,United Kingdom
...,...,...,...,...,...,...,...,...
539091,581433.0,22960.0,JAM MAKING SET WITH JARS,6.0,2011-12-08 15:54:00,4.25,14911.0,EIRE
539731,581439.0,22960.0,JAM MAKING SET WITH JARS,3.0,2011-12-08 16:30:00,8.29,NaN,United Kingdom
539954,581451.0,22960.0,JAM MAKING SET WITH JARS,6.0,2011-12-08 17:57:00,4.25,17144.0,United Kingdom
541049,581492.0,22960.0,JAM MAKING SET WITH JARS,2.0,2011-12-09 10:03:00,8.29,NaN,United Kingdom


In [11]:
df.loc[df['Description'] == 'JAM MAKING SET WITH JARS' ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
21,536368.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 08:34:00,4.25,13047.0,United Kingdom
170,536385.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 09:56:00,4.25,17420.0,United Kingdom
212,536390.0,22960.0,JAM MAKING SET WITH JARS,12.0,2010-12-01 10:19:00,3.75,17511.0,United Kingdom
856,536464.0,22960.0,JAM MAKING SET WITH JARS,1.0,2010-12-01 12:23:00,4.25,17968.0,United Kingdom
866,536466.0,22960.0,JAM MAKING SET WITH JARS,6.0,2010-12-01 12:23:00,4.25,14849.0,United Kingdom
...,...,...,...,...,...,...,...,...
539091,581433.0,22960.0,JAM MAKING SET WITH JARS,6.0,2011-12-08 15:54:00,4.25,14911.0,EIRE
539731,581439.0,22960.0,JAM MAKING SET WITH JARS,3.0,2011-12-08 16:30:00,8.29,NaN,United Kingdom
539954,581451.0,22960.0,JAM MAKING SET WITH JARS,6.0,2011-12-08 17:57:00,4.25,17144.0,United Kingdom
541049,581492.0,22960.0,JAM MAKING SET WITH JARS,2.0,2011-12-09 10:03:00,8.29,NaN,United Kingdom


In [12]:
# Se determina que no es posible de hacer a través de ni mediante StockCode ni
# Description, al no mantenerse el valor de UnitPrice.
# Por lo tanto se eliminan todas las filas donde UnitPrice sea igual o menor a 0

df = df.drop(df[df.UnitPrice <= 0].index)

In [13]:
df.loc[df['UnitPrice'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [14]:
# Se busca identificar filas donde el valor Quantity fuese igual o menor a 0

df.loc[df['Quantity'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1.0,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1.0,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556.0,PLASTERS IN TIN CIRCUS PARADE,-12.0,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984.0,PACK OF 12 PINK PAISLEY TISSUES,-24.0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983.0,PACK OF 12 BLUE PAISLEY TISSUES,-24.0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144.0,ZINC T-LIGHT HOLDER STARS SMALL,-11.0,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1.0,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258.0,VICTORIAN SEWING BOX LARGE,-5.0,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978.0,HANGING HEART JAR T-LIGHT HOLDER,-1.0,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [15]:
# Se eliminan las filas donde el valor Quantity es menor o igual a 0

df = df.drop(df[df.Quantity <= 0].index)

In [16]:
df.loc[df['Quantity'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [17]:
# Se busca generar conversiones de tipos de unidad hacia valores enteros
# en las columnas donde así corresponde

columnas = ['InvoiceNo', 'StockCode', 'Quantity']

for columna in columnas:
  try:
    df[columna] = df[columna].astype(int)
    print(f"La columna '{columna}' fue convertida exitosamente.")
  except:
    print(f"Hay errores en convertir la columna '{columna}'.")


Hay errores en convertir la columna 'InvoiceNo'.
Hay errores en convertir la columna 'StockCode'.
La columna 'Quantity' fue convertida exitosamente.


Lo anterior ocurriría porque existen valores de 'InvoiceNo' y de StockCode' que contienen carácteres.


En el caso de InvoiceNo se deduce que estos caracteres corresponderían a transacciones con alguna clase de especificación o condición especial, pero esta no está detallada ni documentada, por lo que se busca eliminar los carcácteres para que sean tratadas como cualquier otra transacción.


En el caso de StockCode estos se mantienen, debido a que existen varios casos donde el código no es numérico sino que corresponde a una categoría.

In [18]:
# Sintaxis para eliminar caracteres no numéricos de un string:
# df['colB'] = df['colB'].replace(r'\D', r'', regex=True)

try:
  df['InvoiceNo'] = df['InvoiceNo'].replace(r'\D', r'', regex=True)
  print("Se eliminaron exitosamente los caracteres de 'InvoiceNo'.")
  df['InvoiceNo'] = df['InvoiceNo'].astype(int)
  print(f"La columna 'InvoiceNo' fue convertida exitosamente.")
except:
  print(f"Hay errores en convertir la columna 'InvoiceNo'.")

Se eliminaron exitosamente los caracteres de 'InvoiceNo'.
La columna 'InvoiceNo' fue convertida exitosamente.


In [19]:
# Se busca identificar las filas donde hubieran valores nulos

df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132186
Country             0
dtype: int64

In [20]:
df[df['CustomerID'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773.0,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774.0,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786.0,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787.0,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790.0,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150.0,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174.0,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


En esta instancia se identifica que existe una serie de transacciones en las que falta el CustomerID.

El decidir dejarlas en el dataframe generaría errores estadísticos a la hora de generar métricas sobre frecuencia de compras por cliente, al haber varias transacciones por invoice. 

Asimismo, eliminarlas generaría errores estadísticos al evaluar frecuencia de compras de cada producto, al perderse una cantidad significa de registros: 132.186, los que equivalen al 25,18% de las transacciones.

Por lo tanto se decide trabajar dos dataframe: uno para <font color='blue'>**Productos**</font> y otro para <font color='25180125'>**Clientes**</font>.

In [21]:
df_productos = df[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Country']].copy()

In [22]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524878 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    524878 non-null  int64         
 1   StockCode    524878 non-null  object        
 2   Description  524878 non-null  object        
 3   Quantity     524878 non-null  int64         
 4   InvoiceDate  524878 non-null  datetime64[ns]
 5   UnitPrice    524878 non-null  float64       
 6   Country      524878 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 32.0+ MB


In [23]:
df_productos.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

In [24]:
df_productos

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,France
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,France
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,France
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,France


In [25]:
df_clientes = df[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']].copy()

In [26]:
df_clientes = df.dropna(subset=['CustomerID'])

In [27]:
df_clientes.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [28]:
# Se eliminan diplicados de InvoiceNo, tal de que quede solo un registro por Invoice

df_clientes = df_clientes.drop_duplicates(subset=['InvoiceNo'])

In [29]:
df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom
...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom


In [30]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18532 entries, 0 to 541894
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    18532 non-null  int64         
 1   StockCode    18532 non-null  object        
 2   Description  18532 non-null  object        
 3   Quantity     18532 non-null  int64         
 4   InvoiceDate  18532 non-null  datetime64[ns]
 5   UnitPrice    18532 non-null  float64       
 6   CustomerID   18532 non-null  float64       
 7   Country      18532 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.3+ MB


De esta manera se queda con dos dataframes:

<font color='blue'>**Productos**</font>: 524.878 registros.

<font color='green'>**Clientes**</font>: 18.532 registros.

# 1. Productos favoritos

Se buscará determinar cuáles son los productos favoritos.

In [31]:
favoritos = pd.DataFrame(df_productos.groupby('Description')['Quantity'].sum()).reset_index().sort_values('Quantity', ascending=False)

favoritos = favoritos.head(15)

favoritos

,Description,Quantity
2387,"PAPER CRAFT , LITTLE BIRDIE",80995
2052,MEDIUM CERAMIC TOP STORAGE JAR,78033
3934,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54951
1816,JUMBO BAG RED RETROSPOT,48371
3844,WHITE HANGING HEART T-LIGHT HOLDER,37872
2681,POPCORN HOLDER,36749
2337,PACK OF 72 RETROSPOT CAKE CASES,36396
227,ASSORTED COLOUR BIRD ORNAMENT,36362
2741,RABBIT NIGHT LIGHT,30739
2107,MINI PAINT SET VINTAGE,26633


In [32]:
# Para mejorar lectura se cambia la capitalización de las descripciones

favoritos['Description'] = favoritos['Description'].str.capitalize()

favoritos

,Description,Quantity
2387,"Paper craft , little birdie",80995
2052,Medium ceramic top storage jar,78033
3934,World war 2 gliders asstd designs,54951
1816,Jumbo bag red retrospot,48371
3844,White hanging heart t-light holder,37872
2681,Popcorn holder,36749
2337,Pack of 72 retrospot cake cases,36396
227,Assorted colour bird ornament,36362
2741,Rabbit night light,30739
2107,Mini paint set vintage,26633


In [33]:
fig = px.bar(
    favoritos,
    x='Description',
    y='Quantity',
    text='Quantity',
    labels={'Description':'Nombre de producto', 'Quantity':'Cantidad vendida'},
    width=800,
    height=600
    # Tamaño ocupado para exportar a informe:
    # width=1400,
    # height=1000
    )

fig.update_layout(
    title={'text':'Productos favoritos',
           'x':0.5
           },
    xaxis_tickangle=-80
    )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )


fig.show()

# 2. Horario de compras

In [34]:
# Primero, se agrega una columna para determinar la hora de la compra

df_clientes['FechaHora'] = pd.DatetimeIndex(df_clientes['InvoiceDate']).hour

df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FechaHora
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,8
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,8
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,8
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,8
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,8
...,...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom,12
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,12
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,12
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,12


In [35]:
# Luego, se determina a qué día corresponde cada compra

df_clientes['FechaDia'] = df_clientes['InvoiceDate'].dt.dayofweek+1

df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FechaHora,FechaDia
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,8,3
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,8,3
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,8,3
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,8,3
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,8,3
...,...,...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom,12,5
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,12,5
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,12,5
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,12,5


In [36]:
df_clientes['FechaDia'].unique()

array([3, 4, 5, 7, 1, 2])

In [37]:
compras_dia_hora = pd.DataFrame(df_clientes.pivot_table(
    values='InvoiceNo',
    index='FechaHora',
    columns='FechaDia',
    aggfunc='sum'
    )
)

compras_dia_hora

FechaDia,1,2,3,4,5,7
FechaHora,,,,,,
6,NaN,NaN,NaN,563597.0,NaN,NaN
7,2247340.0,2814457.0,2748864.0,4925338.0,3370487.0,NaN
8,47825264.0,64477115.0,65164391.0,66544356.0,67300275.0,NaN
9,140221848.0,162474791.0,161987330.0,164158592.0,148973501.0,1131045.0
10,180014159.0,219521013.0,242388951.0,246662775.0,229201847.0,125567630.0
11,190190784.0,226227323.0,230711821.0,206916627.0,203526291.0,215217733.0
12,268724451.0,297494417.0,341308277.0,327286720.0,259363900.0,257374129.0
13,244211502.0,265349496.0,275787161.0,268270859.0,222323866.0,199601643.0
14,209326125.0,208017567.0,233099767.0,249206353.0,201719717.0,171408506.0


In [38]:
fig = px.imshow(compras_dia_hora,                
                labels=dict(
                    x="Día de la semana",
                    y="Hora del Día",
                    color='InvoiceID'),
                x=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado'],
                text_auto=True,
                aspect="auto",
                color_continuous_scale='blugrn',
                width=800,
                height=600
                # Tamaño ocupado para exportar a informe:
                # width=1400,
                # height=1000
                )

fig.update_layout(xaxis_showgrid=False,
                  yaxis_showgrid=False,
                  coloraxis_colorbar=dict(title='Ventas')
                  )


fig.update_layout(
    title={'text':'Ventas según Día y Hora',
           'x':0.5
           },
    )

fig.show()

# 3. Refuerzo de servidor

Se asume que a diferencia del caso de encontrar la mayor frecuencia de ventas el refuerzo del servidor es algo que se debería hacer con mayor anticipación y no en periodo de días sino semanal, por lo que se buscará evidenciar la frecuencia a una mayor escala.

In [39]:
# Se genera una columna para determinar cada mes

df_clientes['FechaAnioMes'] = pd.to_datetime(df_clientes['InvoiceDate']).dt.to_period('M')

df_clientes['FechaAnioMes'] = df_clientes['FechaAnioMes'].astype(str)


df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FechaHora,FechaDia,FechaAnioMes
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,8,3,2010-12
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,8,3,2010-12
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,8,3,2010-12
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,8,3,2010-12
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,8,3,2010-12
...,...,...,...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom,12,5,2011-12
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,12,5,2011-12
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,12,5,2011-12
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,12,5,2011-12


In [40]:
compras_anio_mes = pd.DataFrame(df_clientes.pivot_table(
    values='InvoiceNo',
    index='FechaAnioMes',
    columns='FechaDia',
    aggfunc='sum'
    )
)

compras_anio_mes

FechaDia,1,2,3,4,5,7
FechaAnioMes,,,,,,
2010-12,111960699,96383362,169895157,196368951,99027178,79595040
2011-01,98060453,96871878,95256617,92572339,84459239,67128879
2011-02,85998589,108202905,105518000,104998554,88683257,48994754
2011-03,115932379,126817142,146584520,154266914,107706075,71096673
2011-04,96192611,119937654,121086571,148054661,95576731,51124820
2011-05,105683645,173762323,147154165,176502971,148874551,108436386
2011-06,112481538,122537523,147551800,188854521,97996753,106315406
2011-07,103058755,121537543,122676095,162536718,125511356,110427753
2011-08,105867874,120567356,152158201,158869151,111008857,72718806


In [41]:
compras_anio_mes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 2010-12 to 2011-12
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1       13 non-null     int64
 1   2       13 non-null     int64
 2   3       13 non-null     int64
 3   4       13 non-null     int64
 4   5       13 non-null     int64
 5   7       13 non-null     int64
dtypes: int64(6)
memory usage: 728.0+ bytes


In [42]:
fig = px.imshow(compras_anio_mes,                
                labels=dict(
                    x="Día de la semana",
                    y="Mes - Año",
                    color='InvoiceID'),
                x=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado'],
                text_auto=True,
                aspect="auto",
                color_continuous_scale='blugrn',
                width=800,
                height=600
                # Tamaño ocupado para exportar a informe:
                # width=1400,
                # height=1000
                )

fig.update_layout(xaxis_showgrid=False,
                  yaxis_showgrid=False,
                  coloraxis_colorbar=dict(title='Ventas'),
                  yaxis_nticks=13
                  )

fig.update_layout(
    title={'text':'Ventas según Día y Mes',
           'x':0.5
           },
    )

fig.show()

Se identificaría asi que son particularmente los meses de Octubre y Noviembre aquellos en los que se debería reforzar el servidor.

# 4. Frecuencia de compra

Ante la pregunta "¿Qué frecuencia de compra tiene un cliente de alta frecuencia" en primer lugar es necesario definir a qué corresponde la "alta frecuencia".

In [43]:
df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FechaHora,FechaDia,FechaAnioMes
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,8,3,2010-12
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,8,3,2010-12
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,8,3,2010-12
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,8,3,2010-12
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,8,3,2010-12
...,...,...,...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777.0,United Kingdom,12,5,2011-12
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,12,5,2011-12
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,12,5,2011-12
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,12,5,2011-12


In [44]:
# Se convierte CustomerID en número entero

try:
  df_clientes['CustomerID'] = df_clientes['CustomerID'].astype(int)
  print(f"Conversión exitosa.")
except:
  print("Error en la conversión.")

Conversión exitosa.


In [45]:
frecuencia_clientes = pd.DataFrame(df_clientes.groupby('CustomerID')['InvoiceNo'].count().reset_index().sort_values('InvoiceNo', ascending=False))

frecuencia_clientes

,CustomerID,InvoiceNo
326,12748,209
1879,14911,201
4010,17841,124
562,13089,97
1661,14606,93
...,...,...
2179,15314,1
2178,15313,1
2175,15308,1
2174,15307,1


In [46]:
frecuencia_clientes.describe()

,CustomerID,InvoiceNo
count,4338.000000,4338.000000
mean,15300.408022,4.272015
std,1721.808492,7.697998
min,12346.000000,1.000000
25%,13813.250000,1.000000
50%,15299.500000,2.000000
75%,16778.750000,5.000000
max,18287.000000,209.000000


In [47]:
fig = px.box(frecuencia_clientes,
             x="InvoiceNo",
             title="Receipts by Payer Gender and Day of Week",
             points="all",
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de frecuencia de compras por usuario/a',
           'x':0.5
           },
    xaxis_title="Cantidad de Compras",
    )

fig.add_vline(x=11, line_width=3, line_dash="dash", line_color="green")

fig.show()

In [48]:
fig = px.box(frecuencia_clientes,
             x="InvoiceNo",
             title="Receipts by Payer Gender and Day of Week",
             points="all",
             log_x=True,
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de frecuencia de compras por usuario/a (escala logarítmica)',
           'x':0.5
           },
    xaxis_title="Cantidad de Compras",
    )

fig.add_vline(x=11, line_width=3, line_dash="dash", line_color="green")

fig.show()

In [49]:
# Desde lo anterior se identifica una distribución que contiene valores atípicos que
# no permiten identificar correctamente un máximo, el que se vuelve necesario calcular

Q1 = frecuencia_clientes['InvoiceNo'].quantile(0.25)
Q3 = frecuencia_clientes['InvoiceNo'].quantile(0.75)

IQR = Q3-Q1
maximo = Q3+(1.5*IQR)

print(f"Valor intercuartil: {IQR}")
print(f"Valor del máximo: {maximo}")

Valor intercuartil: 4.0
Valor del máximo: 11.0


Se determina asi que un cliente de alta frecuencia es aquel que compre más de

In [50]:
# Se determina asi que un cliente de alta frecuencia es uno que ha comprado más de 11 veces

frecuencia_clientes[frecuencia_clientes['InvoiceNo'] >= 11]

,CustomerID,InvoiceNo
326,12748,209
1879,14911,201
4010,17841,124
562,13089,97
1661,14606,93
...,...,...
1807,14810,11
3249,16775,11
1291,14099,11
3413,17001,11


In [51]:
fig = px.box(frecuencia_clientes[frecuencia_clientes['InvoiceNo'] >= 11],
             x="InvoiceNo",
             points="all",
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500,
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de frecuencia de compras de clientes frecuentes (Más de 11 compras)',
           'x':0.5
           },
    xaxis_title="Cantidad de Compras",
    )

fig.add_vline(x=35, line_width=3, line_dash="dash", line_color="green")

fig.show()

In [52]:
fig = px.box(frecuencia_clientes[frecuencia_clientes['InvoiceNo'] >= 11],
             x="InvoiceNo",
             points="all",
             log_x=True,
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500,
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de frecuencia de compras de clientes frecuentes (Más de 11 compras) (escala logarítmica)',
           'x':0.5
           },
    xaxis_title="Cantidad de Compras",
    )

fig.add_vline(x=35, line_width=3, line_dash="dash", line_color="green")

fig.show()

# 5. Gasto

Ante la pregunta "¿Cuánto debería gastar un cliente para considerarlo sobre la media?" es necesario realizar otro dataset para calcular los gastos, debido a que en el de clientes habrían transacciones que fueron eliminadas.

In [53]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [54]:
df_gasto = df[['InvoiceNo', 'Quantity', 'UnitPrice', 'CustomerID']].copy()

df_gasto

,InvoiceNo,Quantity,UnitPrice,CustomerID
0,536365,6,2.55,17850.0
1,536365,6,3.39,17850.0
2,536365,8,2.75,17850.0
3,536365,6,3.39,17850.0
4,536365,6,3.39,17850.0
...,...,...,...,...
541904,581587,12,0.85,12680.0
541905,581587,6,2.10,12680.0
541906,581587,4,4.15,12680.0
541907,581587,4,4.15,12680.0


In [55]:
# Se genera una columna para determinar el gasto total por transacción

df_gasto['GastoTotal'] = ( df_gasto['Quantity'] * df_gasto['UnitPrice'])

df_gasto

,InvoiceNo,Quantity,UnitPrice,CustomerID,GastoTotal
0,536365,6,2.55,17850.0,15.30
1,536365,6,3.39,17850.0,20.34
2,536365,8,2.75,17850.0,22.00
3,536365,6,3.39,17850.0,20.34
4,536365,6,3.39,17850.0,20.34
...,...,...,...,...,...
541904,581587,12,0.85,12680.0,10.20
541905,581587,6,2.10,12680.0,12.60
541906,581587,4,4.15,12680.0,16.60
541907,581587,4,4.15,12680.0,16.60


In [56]:
# Y una matriz que permite determinar el gasto total por cada cliente

gasto_clientes = pd.DataFrame(df_gasto.groupby('CustomerID')['GastoTotal'].sum().reset_index().sort_values('GastoTotal', ascending=False))

gasto_clientes

,CustomerID,GastoTotal
1689,14646.0,280206.02
4201,18102.0,259657.30
3728,17450.0,194390.79
3008,16446.0,168472.50
1879,14911.0,143711.17
...,...,...
3323,16878.0,13.30
4098,17956.0,12.75
3014,16454.0,6.90
1793,14792.0,6.20


In [57]:
gasto_clientes.describe()

,CustomerID,GastoTotal
count,4338.000000,4338.000000
mean,15300.408022,2048.688081
std,1721.808492,8985.230220
min,12346.000000,3.750000
25%,13813.250000,306.482500
50%,15299.500000,668.570000
75%,16778.750000,1660.597500
max,18287.000000,280206.020000


In [58]:
fig = px.box(gasto_clientes,
             x="GastoTotal",
             points="all",
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de gasto total por usuario/a',
           'x':0.5
           },
    xaxis_title="Gasto total",
    )



fig.show()

In [59]:
fig = px.box(gasto_clientes,
             x="GastoTotal",
             points="all",
             log_x=True,
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500,
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de gasto total por usuario/a (escala logarítmica)',
           'x':0.5
           },
    xaxis_title="Gasto total",
    )



fig.show()

In [60]:
mediana_gasto = gasto_clientes['GastoTotal'].mean()

print(f"La media del gasto total es: {mediana_gasto}")

La media del gasto total es: 2048.6880806823424


In [61]:
fig = px.box(gasto_clientes,
             x="GastoTotal",
             points="all",
             log_x=True,
             width=800,
             height=600
             # Tamaño ocupado para exportar a informe:
             #  width=1400,
             #  height=500,
             )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )

fig.update_layout(
    title={'text':'Distribución de gasto total por usuario/a (escala logarítmica)',
           'x':0.5
           },
    xaxis_title="Gasto total",
    )

fig.add_vline(x=mediana_gasto, line_width=3, line_dash="dash", line_color="green")

fig.show()

# 6. Antiguedad

Ante la pregunta "¿Cómo se distribuyen las antigüedades de los clientes?" se buscaría identificar la antiguedad de cada CustomerID

In [62]:
df_clientes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FechaHora,FechaDia,FechaAnioMes
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,8,3,2010-12
7,536366,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,8,3,2010-12
9,536367,84879.0,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,8,3,2010-12
21,536368,22960.0,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047,United Kingdom,8,3,2010-12
25,536369,21756.0,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047,United Kingdom,8,3,2010-12
...,...,...,...,...,...,...,...,...,...,...,...
541865,581583,20725.0,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777,United Kingdom,12,5,2011-12
541867,581584,20832.0,RED FLOCK LOVE HEART PHOTO FRAME,72,2011-12-09 12:25:00,0.72,13777,United Kingdom,12,5,2011-12
541869,581585,22481.0,BLACK TEA TOWEL CLASSIC DESIGN,12,2011-12-09 12:31:00,0.39,15804,United Kingdom,12,5,2011-12
541890,581586,22061.0,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,12,5,2011-12


In [63]:
antiguedad_clientes = pd.DataFrame(df_clientes.groupby('CustomerID')['InvoiceDate'].min())

antiguedad_clientes

,InvoiceDate
CustomerID,
12346,2011-01-18 10:01:00
12347,2010-12-07 14:57:00
12348,2010-12-16 19:09:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00
...,...
18280,2011-03-07 09:52:00
18281,2011-06-12 10:53:00
18282,2011-08-05 13:35:00


In [64]:
# Para determinar la antiguedad se elige como criterio elegir la última fecha 
# en que se envió un Invoice

fecha_maxima = df_clientes['InvoiceDate'].max()
fecha_maxima

Timestamp('2011-12-09 12:50:00')

In [65]:
# Se determina la recencia a partir de la diferencia entre la fecha del primer
# invoice de cada cliente y la fecha máxima

antiguedad_clientes['Antiguedad'] = ( fecha_maxima - (antiguedad_clientes['InvoiceDate'])).dt.days

antiguedad_clientes

,InvoiceDate,Antiguedad
CustomerID,,
12346,2011-01-18 10:01:00,325
12347,2010-12-07 14:57:00,366
12348,2010-12-16 19:09:00,357
12349,2011-11-21 09:51:00,18
12350,2011-02-02 16:01:00,309
...,...,...
18280,2011-03-07 09:52:00,277
18281,2011-06-12 10:53:00,180
18282,2011-08-05 13:35:00,125


In [66]:
# Para graficar distribución se busca cruzar información con Gasto Total

antiguedadgasto_clientes = pd.merge(antiguedad_clientes, gasto_clientes, left_on='CustomerID', right_on='CustomerID', how='left')

antiguedadgasto_clientes

,CustomerID,InvoiceDate,Antiguedad,GastoTotal
0,12346,2011-01-18 10:01:00,325,77183.60
1,12347,2010-12-07 14:57:00,366,4310.00
2,12348,2010-12-16 19:09:00,357,1797.24
3,12349,2011-11-21 09:51:00,18,1757.55
4,12350,2011-02-02 16:01:00,309,334.40
...,...,...,...,...
4333,18280,2011-03-07 09:52:00,277,180.60
4334,18281,2011-06-12 10:53:00,180,80.82
4335,18282,2011-08-05 13:35:00,125,178.05
4336,18283,2011-01-06 14:14:00,336,2045.53


In [67]:
# A lo cual se le suma la cantidad de compras

antiguedadgastofrecuencia_clientes = pd.merge(antiguedadgasto_clientes, frecuencia_clientes, left_on='CustomerID', right_on='CustomerID', how='left')

antiguedadgastofrecuencia_clientes

,CustomerID,InvoiceDate,Antiguedad,GastoTotal,InvoiceNo
0,12346,2011-01-18 10:01:00,325,77183.60,1
1,12347,2010-12-07 14:57:00,366,4310.00,7
2,12348,2010-12-16 19:09:00,357,1797.24,4
3,12349,2011-11-21 09:51:00,18,1757.55,1
4,12350,2011-02-02 16:01:00,309,334.40,1
...,...,...,...,...,...
4333,18280,2011-03-07 09:52:00,277,180.60,1
4334,18281,2011-06-12 10:53:00,180,80.82,1
4335,18282,2011-08-05 13:35:00,125,178.05,2
4336,18283,2011-01-06 14:14:00,336,2045.53,16


In [68]:
fig = px.scatter(antiguedadgastofrecuencia_clientes,
                 x="Antiguedad",
                 y="InvoiceNo",
                 size="GastoTotal",
                 size_max=100,
                 width=1200,
                 height=600
                 # Tamaño ocupado para exportar a informe:
                 #  width=1500,
                 #  height=1000,
)

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.4
    )

fig.update_layout(
    title={'text':'Distribución de antiguedad según frecuencia y gasto total',
           'x':0.5
           },
    xaxis_title="Antiguedad",
    yaxis_title="Frecuencia de compra",
    )



fig.show()

# 7. Segmentación

Ante lo solicitado: "Si se clasifican en 3 grupos a los clientes, en base a su frecuencia de compra ¿Qué diferencias existen entre cada clúster? (Utilice el algoritmo de clusterización Kmeans)."

In [69]:
# En principio se realiza la segmentación mediante kmeans

kmeans = KMeans(n_clusters=3, random_state=0)

In [70]:
# Se renombra la columna de frecuencia, que no se había hecho antes

antiguedadgastofrecuencia_clientes = antiguedadgastofrecuencia_clientes.rename(columns={'InvoiceNo' : 'Frecuencia'})

antiguedadgastofrecuencia_clientes

,CustomerID,InvoiceDate,Antiguedad,GastoTotal,Frecuencia
0,12346,2011-01-18 10:01:00,325,77183.60,1
1,12347,2010-12-07 14:57:00,366,4310.00,7
2,12348,2010-12-16 19:09:00,357,1797.24,4
3,12349,2011-11-21 09:51:00,18,1757.55,1
4,12350,2011-02-02 16:01:00,309,334.40,1
...,...,...,...,...,...
4333,18280,2011-03-07 09:52:00,277,180.60,1
4334,18281,2011-06-12 10:53:00,180,80.82,1
4335,18282,2011-08-05 13:35:00,125,178.05,2
4336,18283,2011-01-06 14:14:00,336,2045.53,16


In [71]:
# antiguedadgastofrecuencia_clientes['Cluster'] = kmeans.fit_predict(antiguedadgastofrecuencia_clientes[['Frecuencia', 'Antiguedad']])
antiguedadgastofrecuencia_clientes['Cluster'] = kmeans.fit_predict(antiguedadgastofrecuencia_clientes[['Frecuencia', 'GastoTotal']])



antiguedadgastofrecuencia_clientes['Cluster'] = antiguedadgastofrecuencia_clientes['Cluster'] + 1
# Se agrega 1 al número de cluster para mejorar legibilidad ante "cliente"

antiguedadgastofrecuencia_clientes['Cluster'].value_counts()

1    4301
2      32
3       5
Name: Cluster, dtype: int64

In [72]:
segmentacion = pd.DataFrame(antiguedadgastofrecuencia_clientes.groupby('Cluster')['CustomerID'].count()).reset_index()

segmentacion['Cluster'] = segmentacion['Cluster'].astype(str)

segmentacion

,Cluster,CustomerID
0,1,4301
1,2,32
2,3,5


In [73]:
segmentacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Cluster     3 non-null      object
 1   CustomerID  3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [74]:
fig = px.bar(
    segmentacion,
    x='Cluster',
    y='CustomerID',
    labels={'Cluster':'Segmento', 'CustomerID':'Cantidad de clientes'},
    width=800,
    height=600
    # Tamaño ocupado para exportar a informe:
    # width=1500,
    # height=600
)

fig.update_layout(
    title={'text':'Segmentación según Frecuencia y Gasto Total',
           'x':0.5
           }
    )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )


fig.show()

A partir de lo anterior se observa que 'Frecuencia' y 'GastoTotal' no son buenos criterios para determinar los segmentos, ya que arroja un resultado demasiado cargado. Se elije como alternativa trabajar con 'Frecuencia' y 'Antiguedad'.

In [75]:
antiguedadgastofrecuencia_clientes = antiguedadgastofrecuencia_clientes.drop('Cluster', axis=1)

antiguedadgastofrecuencia_clientes['Cluster'] = kmeans.fit_predict(antiguedadgastofrecuencia_clientes[['Frecuencia', 'Antiguedad']])

antiguedadgastofrecuencia_clientes['Cluster'] = antiguedadgastofrecuencia_clientes['Cluster'] + 1

antiguedadgastofrecuencia_clientes['Cluster'].value_counts()

1    1754
3    1304
2    1280
Name: Cluster, dtype: int64

In [76]:
segmentacion = pd.DataFrame(antiguedadgastofrecuencia_clientes.groupby('Cluster')['CustomerID'].count()).reset_index()

segmentacion['Cluster'] = segmentacion['Cluster'].astype(str)

segmentacion

,Cluster,CustomerID
0,1,1754
1,2,1280
2,3,1304


In [77]:
segmentacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Cluster     3 non-null      object
 1   CustomerID  3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [78]:
fig = px.bar(
    segmentacion,
    x='Cluster',
    y='CustomerID',
    labels={'Cluster':'Segmento', 'CustomerID':'Cantidad de clientes'},
    width=800,
    height=600
    # Tamaño ocupado para exportar a informe:
    # width=1500,
    # height=600
)

fig.update_layout(
    title={'text':'Segmentación según Frecuencia y Antiguedad',
           'x':0.5
           }
    )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )


fig.show()

In [79]:
antiguedadgastofrecuencia_clientes['Cluster'] = antiguedadgastofrecuencia_clientes['Cluster'].astype(str)

antiguedadgastofrecuencia_clientes

,CustomerID,InvoiceDate,Antiguedad,GastoTotal,Frecuencia,Cluster
0,12346,2011-01-18 10:01:00,325,77183.60,1,1
1,12347,2010-12-07 14:57:00,366,4310.00,7,1
2,12348,2010-12-16 19:09:00,357,1797.24,4,1
3,12349,2011-11-21 09:51:00,18,1757.55,1,2
4,12350,2011-02-02 16:01:00,309,334.40,1,1
...,...,...,...,...,...,...
4333,18280,2011-03-07 09:52:00,277,180.60,1,3
4334,18281,2011-06-12 10:53:00,180,80.82,1,3
4335,18282,2011-08-05 13:35:00,125,178.05,2,2
4336,18283,2011-01-06 14:14:00,336,2045.53,16,1


In [80]:
# antiguedadgastofrecuencia_clientes = antiguedadgastofrecuencia_clientes.drop('Color', axis=1)

In [81]:
# Se busca asignar un color a cada cluster

def Color(x):
  if x == '1':
    return 'rgb(28,82,118)'
  elif x == '2':
    return 'rgb(73,169,171)'
  elif x == '3':
    return 'rgb(25,180,125)'
  else:
    return 'error'

In [82]:
antiguedadgastofrecuencia_clientes['Color'] = antiguedadgastofrecuencia_clientes['Cluster'].apply(lambda x: Color(x))

antiguedadgastofrecuencia_clientes

,CustomerID,InvoiceDate,Antiguedad,GastoTotal,Frecuencia,Cluster,Color
0,12346,2011-01-18 10:01:00,325,77183.60,1,1,"rgb(28,82,118)"
1,12347,2010-12-07 14:57:00,366,4310.00,7,1,"rgb(28,82,118)"
2,12348,2010-12-16 19:09:00,357,1797.24,4,1,"rgb(28,82,118)"
3,12349,2011-11-21 09:51:00,18,1757.55,1,2,"rgb(73,169,171)"
4,12350,2011-02-02 16:01:00,309,334.40,1,1,"rgb(28,82,118)"
...,...,...,...,...,...,...,...
4333,18280,2011-03-07 09:52:00,277,180.60,1,3,"rgb(25,180,125)"
4334,18281,2011-06-12 10:53:00,180,80.82,1,3,"rgb(25,180,125)"
4335,18282,2011-08-05 13:35:00,125,178.05,2,2,"rgb(73,169,171)"
4336,18283,2011-01-06 14:14:00,336,2045.53,16,1,"rgb(28,82,118)"


In [83]:
fig = px.scatter_3d(antiguedadgastofrecuencia_clientes,
                    x='Antiguedad',
                    y='GastoTotal',
                    z='Frecuencia',
                    color='Cluster',
                    symbol='Cluster',
                    color_discrete_sequence=antiguedadgastofrecuencia_clientes['Color'].unique(),
                    opacity=0.7,
                    width=800,
                    height=600
                    # Tamaño ocupado para exportar a informe:
                    # width=1400,
                    # height=1000
                    )

fig.update_traces(
    marker_size = 4
    )

fig.update_layout(scene = dict(
                    xaxis_title='Antiguedad',
                    yaxis_title='Gasto Total',
                    zaxis_title='Frecuencia'),
                    )

fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(210,210,210)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(210,210,210)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="rgb(210,210,210)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",),),
                  )

fig.update_layout(
    title={'text':'Segmentación según Frecuencia y Antiguedad',
           'x':0.5
           }
    )

fig.show()

# 8. Ventas internacionales

Con el fin de complementar los análisis anteriores se busca complementar agregando como variable el país de origen, tal de determinar la importancia de cada uno de estos en las transacciones.

In [84]:
print(f"Número de países: {len(df_productos['Country'].unique())}")

Número de países: 38


In [85]:
df_productos

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,France
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,France
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,France
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,France


In [86]:
compras_paises = df_productos.copy()

In [87]:
compras_paises

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,France
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,France
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,France
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,France


In [88]:
compras_paises['Anio'] = compras_paises['InvoiceDate'].dt.year
compras_paises['Mes'] = compras_paises['InvoiceDate'].dt.month
compras_paises['TotalCompra'] = (compras_paises['Quantity'] * compras_paises['UnitPrice'] )

In [89]:
compras_nacionales = compras_paises.copy()

In [90]:
compras_nacionales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,Anio,Mes,TotalCompra
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom,2010,12,15.30
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,France,2011,12,10.20
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,France,2011,12,12.60
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,France,2011,12,16.60
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,France,2011,12,16.60


In [91]:
# Se busca representar la proporción entre compras nacionales e internacionales

def Sep_Nacionales(x):
  if x == 'United Kingdom':
    return x
  else:
    return 'Internacional'

In [92]:
compras_nacionales['Country'] = compras_nacionales['Country'].apply(lambda x: Sep_Nacionales(x))

In [93]:
compras_nacionales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,Anio,Mes,TotalCompra
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom,2010,12,15.30
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613.0,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,Internacional,2011,12,10.20
541905,581587,22899.0,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,Internacional,2011,12,12.60
541906,581587,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,Internacional,2011,12,16.60
541907,581587,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,Internacional,2011,12,16.60


In [94]:
agrupado_nacionales = pd.DataFrame(compras_nacionales.groupby('Country')['InvoiceNo'].count().reset_index())

agrupado_nacionales

,Country,InvoiceNo
0,Internacional,44893
1,United Kingdom,479985


In [95]:
fig = px.bar(
    agrupado_nacionales,
    x='Country',
    y='InvoiceNo',
    text='InvoiceNo',
    labels={'Country':'País', 'InvoiceNo':'Cantidad de transacciones'},
    width=800,
    height=600
    # Tamaño ocupado para exportar a informe:
    # width=1400,
    # height=1000
    )

fig.update_layout(
    title={'text':'Relación entre ventas Nacionales e Internacionales',
           'x':0.5
           },
    # xaxis_tickangle=90
    )

fig.update_traces(
    marker_color='rgb(59,132,121)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )


fig.show()

In [96]:
# Se genera un dataset para 2010 y otro para 2011

compras_paises_2010 = compras_paises[(compras_paises.Anio == 2010) & (compras_paises.Mes == 12)].copy()

compras_paises_2011 = compras_paises[(compras_paises.Anio == 2011) & (compras_paises.Mes == 12)].copy()

In [97]:
compras_paises_2010.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,Anio,Mes,TotalCompra
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom,2010,12,15.30
1,536365,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom,2010,12,20.34


In [98]:
compras_paises_2011.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,Anio,Mes,TotalCompra
516403,579899,23301.0,GARDENERS KNEELING PAD KEEP CALM,24,2011-12-01 08:33:00,1.65,United Kingdom,2011,12,39.60
516404,579899,22623.0,BOX OF VINTAGE JIGSAW BLOCKS,3,2011-12-01 08:33:00,5.95,United Kingdom,2011,12,17.85
516405,579899,20970.0,PINK FLORAL FELTCRAFT SHOULDER BAG,4,2011-12-01 08:33:00,3.75,United Kingdom,2011,12,15.00
516406,579899,23562.0,SET OF 6 RIBBONS PERFECTLY PRETTY,6,2011-12-01 08:33:00,2.89,United Kingdom,2011,12,17.34
516407,579899,71477.0,COLOURED GLASS STAR T-LIGHT HOLDER,4,2011-12-01 08:33:00,3.95,United Kingdom,2011,12,15.80


In [99]:
# Se determinan gastos totales y frecuencias sumadas

frecuencia_paises_2010 = pd.DataFrame(compras_paises_2010.groupby('Country')['InvoiceNo'].count().reset_index())
gastos_paises_2010 = pd.DataFrame(compras_paises_2010.groupby('Country')['TotalCompra'].sum().reset_index())

frecuencia_paises_2011 = pd.DataFrame(compras_paises_2011.groupby('Country')['InvoiceNo'].count().reset_index())
gastos_paises_2011 = pd.DataFrame(compras_paises_2011.groupby('Country')['TotalCompra'].sum().reset_index())

In [100]:
frecuencia_paises_2010 = frecuencia_paises_2010.rename(columns={'InvoiceNo' : 'Frecuencia2010'})
gastos_paises_2010 = gastos_paises_2010.rename(columns={'TotalCompra' : 'TotalCompra2010'})

frecuencia_paises_2011 = frecuencia_paises_2011.rename(columns={'InvoiceNo' : 'Frecuencia2011'})
gastos_paises_2011 = gastos_paises_2011.rename(columns={'TotalCompra' : 'TotalCompra2011'})

In [101]:
frecuencia_gasto_paises_2010 = pd.merge(frecuencia_paises_2010, gastos_paises_2010, left_on='Country', right_on='Country', how='left')

frecuencia_gasto_paises_2011 = pd.merge(frecuencia_paises_2011, gastos_paises_2011, left_on='Country', right_on='Country', how='left')

frecuencia_gasto_paises = pd.merge(frecuencia_gasto_paises_2010, frecuencia_gasto_paises_2011, left_on='Country', right_on='Country', how='left')

frecuencia_gasto_paises

,Country,Frecuencia2010,TotalCompra2010,Frecuencia2011,TotalCompra2011
0,Australia,31,965.35,NaN,NaN
1,Austria,8,277.20,7.0,683.20
2,Bahrain,1,205.74,NaN,NaN
3,Belgium,95,1809.91,94.0,1417.73
4,Channel Islands,17,363.53,5.0,198.40
5,Cyprus,82,1587.07,NaN,NaN
6,Denmark,20,1281.50,11.0,168.90
7,EIRE,373,10033.26,333.0,7340.89
8,Finland,17,892.80,13.0,1232.76
9,France,434,9616.31,324.0,7276.92


In [102]:
# Se reemplazan todos los NaN por 0

frecuencia_gasto_paises.fillna(value = 0, inplace = True)

frecuencia_gasto_paises

,Country,Frecuencia2010,TotalCompra2010,Frecuencia2011,TotalCompra2011
0,Australia,31,965.35,0.0,0.00
1,Austria,8,277.20,7.0,683.20
2,Bahrain,1,205.74,0.0,0.00
3,Belgium,95,1809.91,94.0,1417.73
4,Channel Islands,17,363.53,5.0,198.40
5,Cyprus,82,1587.07,0.0,0.00
6,Denmark,20,1281.50,11.0,168.90
7,EIRE,373,10033.26,333.0,7340.89
8,Finland,17,892.80,13.0,1232.76
9,France,434,9616.31,324.0,7276.92


In [103]:
frecuencia_gasto_paises.describe()

,Frecuencia2010,TotalCompra2010,Frecuencia2011,TotalCompra2011
count,23.000000,23.000000,23.000000,23.000000
mean,1782.217391,35715.336087,1076.739130,27661.917391
std,8057.298092,154906.806421,4849.195352,122982.414049
min,1.000000,205.740000,0.000000,0.000000
25%,23.000000,852.150000,0.000000,0.000000
50%,49.000000,1661.060000,11.000000,238.000000
75%,105.500000,5769.685000,90.000000,2796.945000
max,38738.000000,746082.220000,23316.000000,591619.740000


In [104]:
frecuencia_gasto_paises.isnull().sum()

Country            0
Frecuencia2010     0
TotalCompra2010    0
Frecuencia2011     0
TotalCompra2011    0
dtype: int64

In [105]:
paises = list(frecuencia_gasto_paises['Country'].values)

paises

['Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Channel Islands',
 'Cyprus',
 'Denmark',
 'EIRE',
 'Finland',
 'France',
 'Germany',
 'Iceland',
 'Italy',
 'Japan',
 'Lithuania',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom']

In [106]:
# Se elimina UK debido a que muestra las ventas nacionales

paises.pop()

'United Kingdom'

In [107]:
countries = paises

data = {"x": [], "y": [], "colors": [], "years": []}

for country in countries:
    data["x"].extend(
        [
            frecuencia_gasto_paises.loc[(frecuencia_gasto_paises.Country == country)]['TotalCompra2010'].values[0],
            frecuencia_gasto_paises.loc[frecuencia_gasto_paises.Country == country]['TotalCompra2011'].values[0],
         None,
        ]
    )
    data["y"].extend([country, country, None]),
    data["colors"].extend(["rgb(73,169,159)", "rgb(28,82,118)", "brown"]),
    data["years"].extend(["2010", "2011", None])

fig = go.Figure(
    data=[
        go.Scatter(
            x=data["x"],
            y=data["y"],
            mode="lines",
            marker=dict(
                color="grey",
            ),
        ),
        go.Scatter(
            x=data["x"],
            y=data["y"],
            mode="markers+text",
            marker=dict(
                color=data["colors"],
                size=10,
            ),
            hovertemplate="""Country: %{y} <br> Total Compras: %{x} <br><extra></extra>""",
        ),
    ]
)

fig.update_layout(
    title={'text':"Crecimiento internacional (Monto total de ventas): Diciembre 2010 - Diciembre 2011",
           'x':0.5
    },
    showlegend=False,
)

fig.update_layout(
    # autosize=False,
    width=600,
    height=800,
    # Tamaño ocupado para exportar a informe:
    # width=1500,
    # height=600,
    xaxis_title="Monto total de ventas",
    yaxis_title="Países",
    margin=dict(
        l=0,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()

In [108]:
countries = paises

data = {"x": [], "y": [], "colors": [], "years": []}

for country in countries:
    data["x"].extend(
        [
            frecuencia_gasto_paises.loc[(frecuencia_gasto_paises.Country == country)]['Frecuencia2010'].values[0],
            frecuencia_gasto_paises.loc[frecuencia_gasto_paises.Country == country]['Frecuencia2011'].values[0],
         None,
        ]
    )
    data["y"].extend([country, country, None]),
    data["colors"].extend(["rgb(73,169,159)", "rgb(28,82,118)", "brown"]),
    data["years"].extend(["2010", "2011", None])

fig = go.Figure(
    data=[
        go.Scatter(
            x=data["x"],
            y=data["y"],
            mode="lines",
            marker=dict(
                color="grey",
            ),
        ),
        go.Scatter(
            x=data["x"],
            y=data["y"],
            mode="markers+text",
            marker=dict(
                color=data["colors"],
                size=10,
            ),
            hovertemplate="""Country: %{y} <br> Total Transacciones: %{x} <br><extra></extra>""",
        ),
    ]
)

fig.update_layout(
    title={'text':"Crecimiento internacional (Cantidad de transacciones): Diciembre 2010 - Diciembre 2011",
           'x':0.5
    },
    showlegend=False,
)

fig.update_layout(
    # autosize=False,
    width=600,
    height=800,
    # Tamaño ocupado para exportar a informe:
    # width=1500,
    # height=600,
    xaxis_title="Cantidad de transacciones",
    yaxis_title="Países",
    margin=dict(
        l=0,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()